<a href="https://colab.research.google.com/github/TaniaHurtado2024/Seminario_BigData/blob/main/1_PPvsSpark_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pandas vs Polars vs Spark vs Dask
[Author: Elias Buitrago Bolivar](https://github.com/ebuitrago?tab=repositories)

Inspired in: https://www.youtube.com/watch?v=mi9f9zOaqM8

Original data: Kaggle

This jupyter notebook is designed to study and compare different tools to read and manipulate data; to be used in the data undertanding phase. The corresponding explanations will be given directly in class, therefore the material isn't autoexplained. Don´t forget ask me for the access to the data. And, please, give credits to the original author's idea and, if consider, also to me.

_Updated: June 20th, 2023_

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Playing with pandas

In [ ]:
import pandas as pd
flights_file1 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2018.parquet"
flights_file2 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2019.parquet"
flights_file3 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2020.parquet"
flights_file4 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2021.parquet"
flights_file5 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2022.parquet"
df1 = pd.read_parquet(flights_file1)
df2 = pd.read_parquet(flights_file2)
df3 = pd.read_parquet(flights_file3)
df4 = pd.read_parquet(flights_file4)
df5 = pd.read_parquet(flights_file5)

KeyboardInterrupt: 

In [ ]:
# df = pd.concat([df3, df5])
# df = df3

In [ ]:
%%timeit

df_agg = df.groupby(['Airline','Year'])[["DepDelayMinutes", "ArrDelayMinutes"]].agg(
    ["mean", "sum", "max"]
)
df_agg = df_agg.reset_index()
df_agg.to_parquet("temp_pandas.parquet")

In [ ]:
!ls -GFlash temp_pandas.parquet

In [ ]:
pd.read_parquet('temp_pandas.parquet')

In [ ]:
pd.read_parquet('temp_pandas.parquet').info()

## Playing with Polars

In [ ]:
import polars as pl

In [ ]:
flights_file1 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2018.parquet"
flights_file2 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2019.parquet"
flights_file3 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2020.parquet"
flights_file4 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2021.parquet"
flights_file5 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2022.parquet"
df1 = pl.scan_parquet(flights_file1)
df2 = pl.scan_parquet(flights_file2)
df3 = pl.scan_parquet(flights_file3)
df4 = pl.scan_parquet(flights_file4)
df5 = pl.scan_parquet(flights_file5)

In [ ]:
 %%timeit

df_polars = (
    pl.concat([df1, df2, df3, df4, df5])
    .groupby(['Airline', 'Year'])
    .agg([
        pl.col("DepDelayMinutes").mean().alias("avg_dep_delay"),
        pl.col("DepDelayMinutes").sum().alias("sum_dep_delay"),
        pl.col("DepDelayMinutes").max().alias("max_dep_delay"),
        pl.col("ArrDelayMinutes").mean().alias("avg_arr_delay"),
        pl.col("ArrDelayMinutes").sum().alias("sum_arr_delay"),
        pl.col("ArrDelayMinutes").max().alias("max_arr_delay"),
      ])
).collect()

df_polars.write_parquet('temp_polars.parquet')

<magic-timeit>:3: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.


11.4 s ± 560 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
!ls -GFlash temp_polars.parquet

12K -rw-r--r-- 1 root 8.1K Jul 11 01:14 temp_polars.parquet


## Playing with PySpark

In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, max, sum, concat

In [ ]:
spark = SparkSession.builder.master("local[1]").appName("airline-example").getOrCreate()

In [ ]:
flights_file1 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2018.parquet"
flights_file2 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2019.parquet"
flights_file3 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2020.parquet"
flights_file4 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2021.parquet"
flights_file5 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2022.parquet"

In [ ]:
df_spark1 = spark.read.parquet(flights_file1)
df_spark2 = spark.read.parquet(flights_file2)
df_spark3 = spark.read.parquet(flights_file3)
df_spark4 = spark.read.parquet(flights_file4)
df_spark5 = spark.read.parquet(flights_file5)

In [ ]:
df_spark = df_spark1.union(df_spark2)
df_spark = df_spark.union(df_spark3)
df_spark = df_spark.union(df_spark4)
df_spark = df_spark.union(df_spark5)

In [ ]:
 %%timeit

df_spark_agg = df_spark.groupby("Airline", "Year").agg(
    avg("ArrDelayMinutes").alias('avg_arr_delay'),
    sum("ArrDelayMinutes").alias('sum_arr_delay'),
    max("ArrDelayMinutes").alias('max_arr_delay'),
    avg("DepDelayMinutes").alias('avg_dep_delay'),
    sum("DepDelayMinutes").alias('sum_dep_delay'),
    max("DepDelayMinutes").alias('max_dep_delay'),
)
df_spark_agg.write.mode('overwrite').parquet('temp_spark.parquet')

8.06 s ± 1.07 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
!ls -GFlash temp_pyspark.parquet

ls: cannot access 'temp_pyspark.parquet': No such file or directory


## Playing with dask

In [2]:
import pandas as pd
import dask.dataframe as dd
flights_file1 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2018.parquet"
flights_file2 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2019.parquet"
flights_file3 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2020.parquet"
flights_file4 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2021.parquet"
flights_file5 = "/content/drive/MyDrive/Data_Flights/Combined_Flights_2022.parquet"
df1 = dd.read_parquet(flights_file1)
df2 = dd.read_parquet(flights_file2)
df3 = dd.read_parquet(flights_file3)
df4 = dd.read_parquet(flights_file4)
df5 = dd.read_parquet(flights_file5)

In [4]:
df = dd.concat([df3, df5])

In [5]:
print(df.compute())

       FlightDate            Airline Origin Dest  Cancelled  Diverted  \
0      2020-09-01        Comair Inc.    PHL  DAY      False     False   
1      2020-09-02        Comair Inc.    PHL  DAY      False     False   
2      2020-09-03        Comair Inc.    PHL  DAY      False     False   
3      2020-09-04        Comair Inc.    PHL  DAY      False     False   
4      2020-09-05        Comair Inc.    PHL  DAY      False     False   
...           ...                ...    ...  ...        ...       ...   
590537 2022-03-31  Republic Airlines    MSY  EWR      False      True   
590538 2022-03-17  Republic Airlines    CLT  EWR       True     False   
590539 2022-03-08  Republic Airlines    ALB  ORD      False     False   
590540 2022-03-25  Republic Airlines    EWR  PIT      False      True   
590541 2022-03-07  Republic Airlines    EWR  RDU      False      True   

        CRSDepTime  DepTime  DepDelayMinutes  DepDelay  ...  WheelsOff  \
0             1905   1858.0              0.0     

In [6]:
df = df.compute()

In [7]:
%%timeit

df_agg = df.groupby(['Airline','Year'])[["DepDelayMinutes", "ArrDelayMinutes"]].agg(
    ["mean", "sum", "max"]
)
df_agg = df_agg.reset_index()
df_agg.to_parquet("temp_dask.parquet")

1.38 s ± 244 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
!ls -GFlash temp_dask.parquet

12K -rw-r--r-- 1 root 9.9K Jul 11 02:26 temp_dask.parquet


In [ ]:
pd.read_parquet('temp_dask.parquet').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   (Airline, )              46 non-null     string 
 1   (Year, )                 46 non-null     int64  
 2   (DepDelayMinutes, mean)  46 non-null     float64
 3   (DepDelayMinutes, sum)   46 non-null     float64
 4   (DepDelayMinutes, max)   46 non-null     float64
 5   (ArrDelayMinutes, mean)  46 non-null     float64
 6   (ArrDelayMinutes, sum)   46 non-null     float64
 7   (ArrDelayMinutes, max)   46 non-null     float64
dtypes: float64(6), int64(1), string(1)
memory usage: 3.0 KB


In [ ]:
pd.read_parquet('temp_dask.parquet')

Airline  Year DepDelayMinutes  \
                                                               mean   
0                 Air Wisconsin Airlines Corp  2020        8.583725   
1                 Air Wisconsin Airlines Corp  2022       13.124801   
2                        Alaska Airlines Inc.  2020        5.818328   
3                        Alaska Airlines Inc.  2022       10.153994   
4                               Allegiant Air  2020       12.825575   
5                               Allegiant Air  2022       22.688601   
6                      American Airlines Inc.  2020        7.624477   
7                      American Airlines Inc.  2022       17.718716   
8                 Capital Cargo International  2020        7.665063   
9                 Capital Cargo International  2022       12.052814   
10                                Comair Inc.  2020       10.068723   
11                                Comair Inc.  2022       16.925615   
12  Commutair Aka Champlain Enterprises, Inc.  2020       12.266858   
13  Commutair Aka Champlain Enterprises, Inc.  2022       16.342795   
14                           Compass Airlines  2020        8.215550   
15                       Delta Air Lines Inc.  2020        5.581694   
16                       Delta Air Lines Inc.  2022       13.842472   
17                       Empire Airlines Inc.  2020        6.861561   
18                          Endeavor Air Inc.  2020        5.653603   
19                          Endeavor Air Inc.  2022       13.284602   
20                                  Envoy Air  2020        6.685444   
21                                  Envoy Air  2022       10.196954   
22                   ExpressJet Airlines Inc.  2020        6.396004   
23                     Frontier Airlines Inc.  2020        7.398021   
24                     Frontier Airlines Inc.  2022       23.560713   
25   GoJet Airlines, LLC d/b/a United Express  2020        8.309550   
26   GoJet Airlines, LLC d/b/a United Express  2022       20.259624   
27                     Hawaiian Airlines Inc.  2020        3.592000   
28                     Hawaiian Airlines Inc.  2022        8.890739   
29                                Horizon Air  2020        4.849894   
30                                Horizon Air  2022        8.893327   
31                            JetBlue Airways  2020        8.213825   
32                            JetBlue Airways  2022       29.101873   
33                         Mesa Airlines Inc.  2020        9.662425   
34                         Mesa Airlines Inc.  2022       18.850643   
35                          Republic Airlines  2020        5.224466   
36                          Republic Airlines  2022       14.896160   
37                      SkyWest Airlines Inc.  2020        8.671988   
38                      SkyWest Airlines Inc.  2022       14.964957   
39                     Southwest Airlines Co.  2020        4.116316   
40                     Southwest Airlines Co.  2022       15.959255   
41                           Spirit Air Lines  2020        7.809889   
42                           Spirit Air Lines  2022       18.395281   
43                      Trans States Airlines  2020       13.739364   
44                      United Air Lines Inc.  2020        6.925499   
45                      United Air Lines Inc.  2022       16.390842   

                       ArrDelayMinutes                     
           sum     max            mean        sum     max  
0     433315.0  1460.0        8.982529   452450.0  1439.0  
1     510581.0  1355.0       13.340409   517261.0  1353.0  
2     772930.0   823.0        6.365082   843157.0   788.0  
3    1278134.0   915.0       11.026280  1382905.0   908.0  
4    1080016.0  1648.0       13.331111  1115734.0  1645.0  
5    1602632.0  1917.0       25.350068  1785963.0  1919.0  
6    4084097.0  3890.0        7.861155  4202644.0  3864.0  
7    8464195.0  2994.0       17.860139  8499122.0  2977.0  
8     512969.0  1482.0       

## Read Results

In [ ]:
import pandas as pd

In [ ]:
#agg_pandas = pd.read_parquet('temp_pandas.parquet')
agg_polars = pd.read_parquet('temp_polars.parquet')
agg_spark = pd.read_parquet('temp_spark.parquet')
agg_dask = pd.read_parquet('temp_dask.parquet')

In [ ]:
#agg_pandas.shape, agg_polars.shape, agg_spark.shape, agg_dask.shape

In [ ]:
#agg_pandas.sort_values(['Airline','Year']).head()

In [ ]:
agg_polars.sort_values(['Airline','Year']).head()

,Airline,Year,avg_dep_delay,sum_dep_delay,max_dep_delay,avg_arr_delay,sum_arr_delay,max_arr_delay
5,Air Wisconsin Airlines Corp,2018,16.753459,1606774.0,1296.0,17.881934,1708887.0,1292.0
97,Air Wisconsin Airlines Corp,2019,16.868511,1742281.0,1690.0,17.610384,1811545.0,1707.0
51,Air Wisconsin Airlines Corp,2020,8.583725,433315.0,1460.0,8.982529,452450.0,1439.0
49,Air Wisconsin Airlines Corp,2021,16.553045,1290194.0,1421.0,17.327440,1346602.0,1416.0
106,Air Wisconsin Airlines Corp,2022,13.124801,510581.0,1355.0,13.340409,517261.0,1353.0


In [ ]:
agg_spark.sort_values(['Airline','Year']).head()

,Airline,Year,avg_arr_delay,sum_arr_delay,max_arr_delay,avg_dep_delay,sum_dep_delay,max_dep_delay
0,Air Wisconsin Airlines Corp,2018,17.881934,1708887.0,1292.0,16.753459,1606774.0,1296.0
48,Air Wisconsin Airlines Corp,2019,17.610384,1811545.0,1707.0,16.868511,1742281.0,1690.0
56,Air Wisconsin Airlines Corp,2020,8.982529,452450.0,1439.0,8.583725,433315.0,1460.0
93,Air Wisconsin Airlines Corp,2021,17.327440,1346602.0,1416.0,16.553045,1290194.0,1421.0
119,Air Wisconsin Airlines Corp,2022,13.340409,517261.0,1353.0,13.124801,510581.0,1355.0


In [ ]:
agg_dask.sort_values(['Airline','Year']).head()

Airline  Year DepDelayMinutes                     \
                                                mean        sum     max   
0  Air Wisconsin Airlines Corp  2020        8.583725   433315.0  1460.0   
1  Air Wisconsin Airlines Corp  2022       13.124801   510581.0  1355.0   
2         Alaska Airlines Inc.  2020        5.818328   772930.0   823.0   
3         Alaska Airlines Inc.  2022       10.153994  1278134.0   915.0   
4                Allegiant Air  2020       12.825575  1080016.0  1648.0   

  ArrDelayMinutes                     
             mean        sum     max  
0        8.982529   452450.0  1439.0  
1       13.340409   517261.0  1353.0  
2        6.365082   843157.0   788.0  
3       11.026280  1382905.0   908.0  
4       13.331111  1115734.0  1645.0